In [2]:
%%bash
sudo apt-get -qq update
sudo apt-get -y install --no-install-recommends \
    tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd \
    libtesseract-dev build-essential cmake

python -m pip install -U pip wheel --quiet

PYPI=https://pypi.org/simple
pip install --no-cache-dir --index-url $PYPI \
    pillow pytesseract==0.3.10 \
    "opencv-python-headless>=4.11,<5" \
    networkx==3.3 \
    sentence-transformers==2.7.0 transformers==4.41.0 \
    torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 \
    git+https://github.com/facebookresearch/segment-anything.git --no-deps \
    gradio==4.25.0



Reading package lists...
Building dependency tree...
Reading state information...
build-essential is already the newest version (12.9ubuntu3).
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
tesseract-ocr-eng set to manually installed.
tesseract-ocr-osd is already the newest version (1:4.00~git30-7274cfa-1.1).
tesseract-ocr-osd set to manually installed.
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
The following additional packages will be installed:
  libarchive-dev libleptonica-dev
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev
0 upgraded, 3 newly installed, 0 to remove and 37 not upgraded.
Need to get 3,743 kB of archives.
After this operation, 16.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.4 [581 kB]
Get:2 http://archive.ubuntu

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-2zjdjark
ERROR: Could not find a version that satisfies the requirement pytesseract==0.3.10 (from versions: none)
ERROR: No matching distribution found for pytesseract==0.3.10


CalledProcessError: Command 'b'# 0\xef\xb8\x8f\xe2\x83\xa3 System OCR\nsudo apt-get -qq update\nsudo apt-get -y install --no-install-recommends \\\n    tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd \\\n    libtesseract-dev build-essential cmake\n\n# 1\xef\xb8\x8f\xe2\x83\xa3 fresh pip\npython -m pip install -U pip wheel --quiet\n\n# 2\xef\xb8\x8f\xe2\x83\xa3 Python wheels \xe2\x80\x94 all cp311-ready\nPYPI=https://pypi.org/simple\npip install --no-cache-dir --index-url $PYPI \\\n    pillow pytesseract==0.3.10 \\\n    "opencv-python-headless>=4.11,<5" \\\n    networkx==3.3 \\\n    sentence-transformers==2.7.0 transformers==4.41.0 \\\n    torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 \\\n    git+https://github.com/facebookresearch/segment-anything.git --no-deps \\\n    gradio==4.25.0\n\n'' returned non-zero exit status 1.

In [5]:
pip install segment_anything pytesseract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pytesseract]


In [4]:
pip install gradio

In [3]:
pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 108.7 MB/s eta 0:00:00


In [6]:
!wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth


In [7]:
!sha256sum sam_vit_b_01ec64.pth

ec2df62732614e57411cdcf32a23ffdf28910380d03139ee0f4fcbe91eb8c912  sam_vit_b_01ec64.pth


In [8]:
import os, re, json, yaml, logging, math, shutil, mimetypes, io
from typing import List, Tuple

import cv2, numpy as np, networkx as nx, gradio as gr
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from segment_anything import sam_model_registry, SamPredictor
import pytesseract

try:
    import faiss; _HAS_FAISS=True
except ModuleNotFoundError:
    from sklearn.neighbors import NearestNeighbors; _HAS_FAISS=False

DEBUG = bool(os.getenv('DEBUG_SWIMLANE'))
logging.basicConfig(level=logging.DEBUG if DEBUG else logging.INFO,
                    format='[%(levelname)s] %(message)s')

# 0  OCR & models

if not shutil.which('tesseract'):
    raise RuntimeError('Install tesseract‑ocr')

pytesseract.pytesseract.tesseract_cmd = shutil.which('tesseract')

_SAM_CKPT = os.getenv('SAM_CKPT', 'sam_vit_b_01ec64.pth')
if not os.path.exists(_SAM_CKPT):
    raise RuntimeError('Download sam_vit_b_01ec64.pth')

sam = sam_model_registry['vit_b'](_SAM_CKPT)
sam_predictor = SamPredictor(sam)
qa_model = pipeline('question-answering', model='deepset/roberta-base-squad2')
embedder  = SentenceTransformer('all-mpnet-base-v2')


# 1  Static docs (domain‑specific)
doc_chunks = [
    'To place an order, the customer submits a request via the UI.',
    'The Sales team confirms receipt and checks stock.',
    'If not in stock, the order is canceled.',
    'If in stock, the system checks credit card validity before payment.',
    'Finance handles payment and initiates order delivery.'
]
_doc_emb = embedder.encode(doc_chunks, convert_to_numpy=True)
if _HAS_FAISS:
    _doc_idx = faiss.IndexFlatL2(_doc_emb.shape[1]); _doc_idx.add(_doc_emb)
else:
    _doc_idx = NearestNeighbors(metric='euclidean').fit(_doc_emb)


# 2  Swim‑lane helper functions
ALIAS = {'submit order':'place order','credit card invalid':'cancel order'}

def _norm(t:str):
    t=re.sub(r'[^a-z0-9 ]','',t.lower().strip()); return ALIAS.get(t,t) or 'unknown'

def _ocr(img):
    return re.sub(r'\s+',' ',pytesseract.image_to_string(img,config='--psm 6')).strip()

def _sam_boxes(img: np.ndarray, min_area=1000):
    sam_predictor.set_image(img)
    H,W,_=img.shape
    pts=np.array([[x,y] for y in np.linspace(0,H-1,10) for x in np.linspace(0,W-1,10)],dtype=np.float32)
    labels=np.ones(len(pts),dtype=np.int32)
    masks,_,_=sam_predictor.predict(pts,labels,multimask_output=False)
    if masks is None or masks.size==0: return [(0,0,W,H)]
    mask=(masks[0]*255).astype('uint8'); mask=cv2.medianBlur(mask,5)
    num,labels_arr,stats,_=cv2.connectedComponentsWithStats(mask,8)
    boxes=[]
    for i in range(1,num):
        x,y,w,h,area=stats[i]
        if area<min_area or area>0.8*H*W: continue
        x,y,w,h=max(0,x),max(0,y),min(w,W-x),min(h,H-y)
        if w>0 and h>0: boxes.append((x,y,w,h))
    return boxes or [(0,0,W,H)]

def _detect_lines(img):
    e=cv2.Canny(cv2.cvtColor(img,cv2.COLOR_BGR2GRAY),50,150)
    l=cv2.HoughLinesP(e,1,np.pi/180,60,minLineLength=40,maxLineGap=5)
    return [] if l is None else [tuple(x) for x in l[:,0]]

_center=lambda b:(b[0]+b[2]/2,b[1]+b[3]/2)
_dist=lambda p,q: math.hypot(p[0]-q[0],p[1]-q[1])

def build_graph(img):
    boxes=_sam_boxes(img); G=nx.DiGraph()
    for i,(x,y,w,h) in enumerate(boxes):
        txt=_norm(_ocr(img[y:y+h,x:x+w])) or f'node_{i}'; G.add_node(i,text=txt,bbox=(x,y,w,h))
    for x1,y1,x2,y2 in _detect_lines(img):
        tail,head=(x1,y1),(x2,y2)
        src=min(G.nodes,key=lambda n:_dist(tail,_center(G.nodes[n]['bbox'])))
        dst=min(G.nodes,key=lambda n:_dist(head,_center(G.nodes[n]['bbox'])))
        if src!=dst and not G.has_edge(src,dst): G.add_edge(src,dst)
    if DEBUG:
        print('\nNODES:'); [print(f" {n}: {d['text']} bbox={d['bbox']}") for n,d in G.nodes(data=True)]
        print('EDGES:');  [print(f" {u}->{v}") for u,v in G.edges()]
    return G

def _succ(G, ref):
    for n,d in G.nodes(data=True):
        if ref in d['text']:
            succ=list(G.successors(n)); idx=d['bbox'][1]
            if not succ: return ''
            best=min(succ,key=lambda s:(G.nodes[s]['bbox'][1]-idx if G.nodes[s]['bbox'][1]>idx else float('inf')))
            return G.nodes[best]['text']
    return ''

def _top_k(q_emb,flow_emb,k=4):
    if _HAS_FAISS:
        idx=faiss.IndexFlatL2(flow_emb.shape[1]); idx.add(np.vstack([_doc_emb,flow_emb])); _,I=idx.search(q_emb,k); return I[0]
    emb=np.vstack([_doc_emb,flow_emb]); nn=NearestNeighbors(n_neighbors=k).fit(emb); _,I=nn.kneighbors(q_emb,k); return I[0]

_after_r=re.compile(r'after (?:the )?(.*)',re.I)

def answer(img,q):
    G=build_graph(img)
    if m:=_after_r.search(q):
        nxt=_succ(G,_norm(m.group(1)));  return nxt.capitalize() if nxt else 'Sorry, could not answer.'
    flow='; '.join(d['text'] for _,d in G.nodes(data=True))
    f_emb=embedder.encode([flow],convert_to_numpy=True); q_emb=embedder.encode([q],convert_to_numpy=True)
    ctx='\n'.join((doc_chunks+[flow])[i] for i in _top_k(q_emb,f_emb))
    return qa_model(question=q,context=ctx)['answer'].capitalize()

# 3  New helpers for non‑image docs

def _openapi_to_str(spec: dict)->str:
    out=[]
    for path,pi in spec.get('paths',{}).items():
        for verb,val in pi.items():
            if isinstance(val,dict): out.append(f"{verb.upper()} {path} — {val.get('summary','')}")
    return "\n".join(out) or json.dumps(spec)[:2000]

def _rag(question:str, context:str)->str:
    ans=qa_model(question=question, context=context).get('answer',''); return ans.capitalize() or 'Sorry, could not answer.'

# 4  Multimodal router

def router(image, doc_file, question):
    if not question.strip():
        return 'Ask a question.'

    if image is not None:
        return answer(image, question)

    if doc_file is None:
        return 'Upload a diagram or a documentation file.'

    file_path = str(doc_file)
    ext = os.path.splitext(file_path)[1].lower()

    # Error Handling
    if ext in {'.png', '.jpg', '.jpeg', '.bmp', '.gif'}:
        return 'Please use the first (image) input for PNG/JPG diagrams.'

    if hasattr(doc_file, 'read'):
        data = doc_file.read()
    else:
        with open(file_path, 'rb') as fh:
            data = fh.read()

    # For OpenAPI YAML / JSON
    if ext in {'.yaml', '.yml', '.json'}:
        try:
            spec = yaml.safe_load(data)
        except Exception:
            spec = json.loads(data)
        context = _openapi_to_str(spec)
        return _rag(question, context)

    try:
        text = data.decode('utf-8', errors='ignore')
    except Exception:
        text = str(data)
    return _rag(question, text[:4000])

    try: txt=data.decode('utf-8',errors='ignore')
    except Exception: txt=str(data)
    return _rag(question, txt[:4000])

# Step 5  Gradio UI

gr.Interface(
             fn=router,
             inputs=[
                 gr.Image(type='numpy', label='Swim‑lane diagram (PNG/JPG)'),
                 gr.File(label='OpenAPI YAML/JSON or Text'),
                 gr.Textbox(label='Question')
             ],
             outputs=gr.Textbox(label='Answer'),
             title='Multimodal Dev‑Portal Chatbot',
             description='Upload a swim‑lane image *or* a documentation file (OpenAPI YAML/JSON, plain text) and ask a question about the workflow.'
).launch(debug=DEBUG)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b361091c0ff44cf6e8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
